## 로컬 LLM 사용


이 실습에서는 로컬 LLM(Local Language Model)을 사용하여 문장을 생성하는 방법을 알아보겠습니다.

Langchain을 이용하면 간단한 코드로 로컬 LLM을 사용할 수 있습니다.


### 1. 텍스트 생성하기


**필요한 라이브러리 불러오기**


In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

**Ollama를 이용한 llama3.1 불러오기**

Ollama는 다양한 LLM 모델을 쉽게 사용할 수 있도록 도와주는 라이브러리입니다.

이 Ollama를 이용하여 llama3.1 모델을 불러옵니다.


In [3]:
sudo ollama pull llama3.1

SyntaxError: invalid syntax (48136590.py, line 1)

In [6]:
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
Error: open /mnt/elice/dataset/models/blobs/sha256-667b0c1932bc6ffc593ed1d03f895bf2dc8dc6df21db3042284a6f4416b06a29-partial-0: read-only file system


Langchain은 Ollama의 LLM을 사용하는 모듈도 제공하고 있습니다.


In [3]:
llm = ChatOllama(model="llama3.1")  # 사용할 모델

# 사용자 입력을 받아 AI에게 전달하고 AI의 응답 중 텍스트만 출력
chain = llm | StrOutputParser()

이제 `chain`을 이용하면 쉽게 로컬 LLM을 사용할 수 있습니다.

`chain`에 문장을 입력하여 텍스트를 생성해 보겠습니다.


In [ ]:
user_input = input("You: ")
resp = chain.invoke(user_input)
print("AI:", resp)

`chain`의 마지막에 StrOutputParser를 연결했기 때문에, 출력 결과에서 텍스트만 자동으로 추출됩니다.


이번에는 `StrOutputParser`를 뺀 `chain2`를 사용하여 출력 결과를 확인해 보겠습니다.


In [11]:
chain2 = llm  # StrOutputParser()를 사용하지 않으면 AI의 응답을 그대로 출력

In [ ]:
resp = chain2.invoke(user_input)
print("You:", user_input)
print(resp)

ChatGPT의 API를 사용할 때와 마찬가지로, 다양한 정보가 포함된 출력 결과를 확인할 수 있습니다.


### 2. 로컬 LLM을 이용한 챗봇


이번에는 위에서 사용해본 로컬 LLM을 이용하여 간단한 챗봇을 만들어보겠습니다.

주석을 참고하여 대략 어떻게 정보가 순서대로 전달되는지 확인해보세요.


In [37]:
messages_with_variables = [
    (
        "system",
        "당신은 마케터를 위한 친절한 지원 챗봇입니다.",
    ),  # chatgpt의 system role과 동일한 역할
    ("human", "{question}."),  # chatgpt의 user role과 동일한 역할
]
prompt = ChatPromptTemplate.from_messages(
    messages_with_variables
)  # 정해진 메시지를 채워넣는 템플릿
parser = StrOutputParser()  # AI의 응답 중 텍스트만 출력
chain = prompt | llm | parser  # 템플릿 -> AI -> 텍스트 출력

이제 질문을 전달해보겠습니다.


In [ ]:
print(chain.invoke("키오스크 관련 설문조사 결과를 알려줘"))

매번 다른 대답이 나오지만, 출처를 알 수 없거나 무작위로 생성된 대답이 나올 수 있습니다.

이런 현상 역시 할루시네이션의 일종으로 볼 수 있습니다.


### 3. 데이터 기반 챗봇


이를 해결하기 위해서는 답변에 사용할 데이터를 함께 프롬프트로 제공해야 합니다.

시장조사 PDF 문서를 불러와 보겠습니다.

- 한국소비자원의 2022년 키오스크(무인정보단말기) 이용 실태조사 보고서를 활용했습니다
  - https://www.kca.go.kr/smartconsumer/sub.do?menukey=7301&mode=view&no=1003409523&page=2&cate=00000057
- 이 실태조사 보고서는 2022년 키오스크의 사용자 경험, 접근성, 후속 조치에 대해 논의하는 보고서입니다.
- 이를 활용해서 키오스크를 어떻게 세일즈 할 수 있을지 아이디어를 제공하는 챗봇을 만들어야 하는 상황이라고 가정해 봅시다.


In [16]:
from langchain.document_loaders import PyPDFLoader

doc_path = "키오스크(무인정보단말기) 이용실태 조사.pdf"
loader = PyPDFLoader(doc_path)
docs = loader.load()

이 보고서는 내용이 꽤 긴 보고서입니다.


In [ ]:
doc_len = [len(doc.page_content) for doc in docs]
print(f"Page 수: {len(docs)}")
print(f"각 Document 별 길이: {doc_len}")

페이지에 "설문" 이라는 단어가 포함된 페이지만 추출하여 사용해보겠습니다.


In [ ]:
data = []

for doc in docs:
    if "설문" in doc.page_content:
        data.append(doc.page_content)

print(f"설문이 포함된 Page 수: {len(data)}")

이제 이 데이터를 이용하여 챗봇을 만들어보겠습니다.


In [44]:
context = "\n".join(data)

messages_with_contexts = [
    (
        "system",
        "당신은 마케터를 위한 친절한 지원 챗봇입니다. 사용자가 입력하는 정보를 바탕으로 질문에 답하세요.",
    ),
    (
        "human",
        "정보: " + context + ".\n{question}",
    ),  # pdf의 내용을 context로 사용
]

prompt = ChatPromptTemplate.from_messages(
    messages_with_contexts
)  # 정해진 메시지를 채워넣는 템플릿
parser = StrOutputParser()  # AI의 응답 중 텍스트만 출력
chain = prompt | llm | parser  # 템플릿 -> AI -> 텍스트 출력

In [ ]:
resp = chain.invoke("키오스크 설문조사 결과를 알려줘")
print(resp)

결과가 그다지 개선된 것 같지는 않습니다.

이는 모델의 성능이 좋지 않거나, 전달된 프롬프트가 적절하지 않아서 그럴 수 있습니다.


### 4. 모델 교체


이번에는 앞에서 다뤄보았던 OpenAI의 GPT 모델을 사용해보겠습니다.

Langchain은 다양한 LLM 모델을 사용할 수 있도록 지원하고 있습니다.

LLM을 바꾸는 것도 간단합니다.


In [56]:
llm = ChatOpenAI(
    api_key="sk-",  # API키
    model="gpt-4o-mini",  # 사용할 모델
)

# 체인 다시 정의
chain = prompt | llm | parser  # 템플릿 -> AI -> 텍스트 출력

이제 `chain`의 `llm`이 chatGPT로 바뀌었습니다.

같은 프롬프트와 질문으로 응답을 생성해 봅니다.


In [ ]:
resp = chain.invoke("키오스크 설문조사 결과를 알려줘")
print(resp)

로컬 LLM을 사용할 때와는 다르게, ChatGPT는 더 자연스러운 대화를 생성하는 것을 확인할 수 있습니다.
